<a href="https://colab.research.google.com/github/buganart/melgan-neurips/blob/master/melgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V1000
!nvidia-smi -L

In [ ]:
#@markdown Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
from google.colab import output
!pip install wandb==0.10.8
clear_on_success()
!wandb login
clear_on_success("Ok! Logged into wandb!")

In [ ]:
#@markdown Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
audio_db_dir = "/content/drive/My Drive/AUDIO DATABASE/TESTING" #@param {type:"string"}
experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}
melgan_output_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan-outputs" #@param {type:"string"}

audio_db_dir = Path(audio_db_dir)
melgan_output_dir = Path(melgan_output_dir)
experiment_dir = Path(experiment_dir)

for path in [experiment_dir, melgan_output_dir]:
    path.mkdir(parents=True, exist_ok=True)

if not audio_db_dir.exists():
    raise RuntimeError("audio_db_dir {audio_db_dir} does not exists.")

In [ ]:
!git clone https://github.com/buganart/melgan-neurips

In [ ]:
%cd /content/melgan-neurips
%pip install -r requirements.txt
clear_on_success("Dependencies installed.")

Below: change /MYDIRECTORY to the desired directory name


In [ ]:
local_wav_dir = Path("/content/wavs/")
!rsync -avP "$audio_db_dir"/ "$local_wav_dir"/

In [ ]:
!ls "$local_wav_dir"/*.wav | head -n 100 > train_files.txt
!ls "$local_wav_dir"/*.wav | tail -n 80 > test_files.txt
print("Test")
!cat test_files.txt

print("Train")
!cat train_files.txt

In [ ]:
# This done a bit weirdly because setting PYTHONPATH=$PWD removes variables afterwards. A colab bug, maybe.
!env PYTHONPATH="$(pwd)" python scripts/train.py --save_path "$experiment_dir" --data_path . 

In [ ]:
#@markdown Copy the run directory from the beginning of the cell above. 

#@markdown The line looks something like

#@markdown `wandb: Run data is saved locally in /content/drive/.../melgan/wandb/run-20201030_030400-21bhoeww`

#@markdown and we only need the part after the last slash.

run_dir_name = "run-20201030_030400-21bhoeww" #@param {type:"string"}

run_dir = experiment_dir / "wandb" / run_dir_name
export_dir = melgan_output_dir / run_dir_name
melgan_args = run_dir / "files" / "args.yml"
melgan_netG = run_dir / "files" / "best_netG.pt" 

!cp -v "$melgan_args" "$export_dir"
!cp -v "$melgan_netG" "$export_dir"
!cp -v "/content/melgan-neurips/mel2wav/modules.py" "$export_dir"

print()
print(f"Melgan files saved in {export_dir}.")